In [1]:
import gymnasium as gym
import time
import pygame
from stable_baselines3 import PPO

import config
from wrappers import CliffWalkingStateWrapper

def watch_agent():
    raw_env = gym.make(config.ENV_ID, render_mode="human")
    env = CliffWalkingStateWrapper(raw_env)

    try:
        model = PPO.load("ppo_cliffwalking_rlhf")
        print("Loaded trained PPO model.")
    except FileNotFoundError:
        print("Error: Could not find 'ppo_cliffwalking_rlhf.zip'. Did you run main.py?")
        return

    obs, _ = env.reset()
    print("Starting simulation...")
    print("Press [ESC] to quit episode/simulation.")

    try:
        for episode in range(5):
            print(f"--- Episode {episode + 1} ---")
            done = False
            total_steps = 0
            
            while not done:
                for event in pygame.event.get():
                    if event.type == pygame.QUIT:
                        raise KeyboardInterrupt
                    if event.type == pygame.KEYDOWN:
                        if event.key == pygame.K_ESCAPE:
                            print("\nESC pressed. Exiting...")
                            raise KeyboardInterrupt

                action, _ = model.predict(obs, deterministic=True)
                action_int = action.item() 
                
                obs, reward, terminated, truncated, info = env.step(action_int)
                
                done = terminated or truncated
                total_steps += 1
                
                time.sleep(0.05) 
            
            print(f"Episode finished in {total_steps} steps.")
            obs, _ = env.reset()
            
    except KeyboardInterrupt:
        print("\nStopped by user.")
    finally:
        env.close()
        pygame.quit()

if __name__ == "__main__":
    watch_agent()

c:\Users\ashut\anaconda3\envs\PbRL_env\Lib\site-packages\pygame\pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists
c:\Users\ashut\anaconda3\envs\PbRL_env\Lib\site-packages\stable_baselines3\common\on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


Loaded trained PPO model.
Starting simulation...
Press [ESC] to quit episode/simulation.
--- Episode 1 ---
Episode finished in 15 steps.
--- Episode 2 ---
Episode finished in 15 steps.
--- Episode 3 ---

ESC pressed. Exiting...

Stopped by user.
